# import pandas as pd

### Import libraries

In [4]:
import pandas as pd
import numpy as np
import os
import tarfile
import urllib.request
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
import joblib
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

In [5]:
DOWNLOAD_ROOT = "https://github.com/ageron/handson-ml/raw/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [6]:
fetch_housing_data()

### Load dataset

In [7]:

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [8]:
housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


## 1-Target attribute are “median_income”. 


In [9]:
# first i need to check the shape of data , missing values of data and then make median income a target features 
#and all others features are independent features.
print("the shape of housing dataset is ",housing.shape)
print("the info of each columns of dataset is ", housing.info())



the shape of housing dataset is  (20640, 10)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB
the info of each columns of dataset is  None


In [10]:
print("the statistics of housing data is ")
housing.describe()

the statistics of housing data is 


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


In [11]:
# i also check the missing values in the housing dataset for good results.
# handling of missing values is very important because it make huge impact on prediction.
print("the missing values in housing dataset is ")
print(housing.isnull().sum())
print("i handle the missing values of total bedrooms by interpolate method")
housing=housing.interpolate(method ='linear', limit_direction ='forward')
housing.isnull().sum()

the missing values in housing dataset is 
longitude               0
latitude                0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
ocean_proximity         0
dtype: int64
i handle the missing values of total bedrooms by interpolate method


longitude             0
latitude              0
housing_median_age    0
total_rooms           0
total_bedrooms        0
population            0
households            0
median_income         0
median_house_value    0
ocean_proximity       0
dtype: int64

In [12]:
housing.columns

Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_house_value', 'ocean_proximity'],
      dtype='object')

## 2-Use StratifiedShuffleSplit to split the data into 80% training and 20% testing. 

The stratified-shuffle-split will be based on the categories of “median_house_value”.  Please first categorize “median_house_value” into 3 reasonable categories before splitting.


In [13]:
# i want to calculate the count of each value in median house value columns because i want to convert it into categories.
housing['median_house_value'].value_counts()

500001.0    965
137500.0    122
162500.0    117
112500.0    103
187500.0     93
           ... 
359200.0      1
54900.0       1
377600.0      1
81200.0       1
47000.0       1
Name: median_house_value, Length: 3842, dtype: int64

In [14]:
# in this block i convert median_house_value in category
for i in housing['median_house_value']:
    if i<=150000:       #less than 150000 house value is categorized as low
        housing['median_house_value'].replace(i,'Low',inplace=True)
    elif i>150000 and i<=300000:   #between 150000 and 3000000 house value is categorized as medium      
        housing['median_house_value'].replace(i,'medium',inplace=True)
    elif i>300000:          #greater than 300000 house value is categorized as high.
        housing['median_house_value'].replace(i,'high',inplace=True)

In [15]:
#counts the value in all categorize of house median value column
housing['median_house_value'].value_counts()

medium    9184
Low       7620
high      3836
Name: median_house_value, dtype: int64

In [16]:
housing["median_house_cat"] = housing['median_house_value'].copy()

In [17]:
# i will split the data in stratified shuffle split 
#Stratified ShuffleSplit cross-validator. Provides train/test indices to split data in train/test sets. 
#This cross-validation object is a merge of StratifiedKFold and ShuffleSplit, which returns stratified randomized folds
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing['median_house_value']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [18]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("median_house_value", axis=1, inplace=True)

In [19]:
# dependent and independent feature
housing = strat_train_set.drop("median_income", axis=1) 
housing_labels = strat_train_set["median_income"]

In [20]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,ocean_proximity,median_house_cat
8555,-118.40,33.92,25.0,1453.0,271.0,695.0,283.0,<1H OCEAN,high
14614,-117.17,32.79,43.0,1269.0,297.0,946.0,285.0,NEAR OCEAN,Low
363,-122.14,37.75,36.0,690.0,105.0,299.0,109.0,NEAR BAY,medium
5973,-117.77,34.12,15.0,4260.0,770.0,2007.0,695.0,INLAND,medium
10357,-117.66,33.59,18.0,4552.0,706.0,1918.0,671.0,<1H OCEAN,medium


In [21]:
print(housing_labels)

8555     5.9499
14614    2.1447
363      4.0313
5973     4.4609
10357    7.5791
          ...  
16854    4.8000
716      2.4042
12661    2.8139
7942     4.6765
8627     5.9683
Name: median_income, Length: 16512, dtype: float64


In [22]:
#handle categorical features(ocean_proximity,median_house_cat) by label encoding method.
#Label Encoding is a popular encoding technique for handling categorical variables. In this technique, 
#each label is assigned a unique integer based on alphabetical ordering
from sklearn.preprocessing import LabelEncoder
housing['ocean_proximity'] = LabelEncoder().fit_transform(housing['ocean_proximity'])
housing['median_house_cat']=LabelEncoder().fit_transform(housing['median_house_cat'])

In [23]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,ocean_proximity,median_house_cat
8555,-118.40,33.92,25.0,1453.0,271.0,695.0,283.0,0,1
14614,-117.17,32.79,43.0,1269.0,297.0,946.0,285.0,4,0
363,-122.14,37.75,36.0,690.0,105.0,299.0,109.0,3,2
5973,-117.77,34.12,15.0,4260.0,770.0,2007.0,695.0,1,2
10357,-117.66,33.59,18.0,4552.0,706.0,1918.0,671.0,0,2


In [24]:
# Data normalization
#some feature have value in higher domain so need to scale and shrink the features for better results
from sklearn.preprocessing import StandardScaler
sc=StandardScaler(with_mean=False)
housing_prepaired=sc.fit_transform(housing)

## 3-Examine the following machine learning models: 

Linear Regression

Decision Tree Regression

Random Forest Regression

Support Vector Machine Regression. 


(The code to create an instance for Support Vector Machine is:  svm_reg = SVR()).  Your examination must be based on both how each model fits the training data and how each model performs based on 10 cross validations.

In [25]:
#print mean , SD for cross validation
def display_scores(scores):
    print("Scores of 10 CV:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

### Linear Regression

In [26]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepaired, housing_labels)

LinearRegression()

**Evluation metrics **

In [27]:
from sklearn.metrics import mean_squared_error,r2_score
housing_predictions = lin_reg.predict(housing_prepaired)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
print("mean squared error of linear regression is",lin_mse)
lin_r2_score=r2_score(housing_labels, housing_predictions)
print("R Squared score of linear regression is",lin_r2_score)
lin_rmse = np.sqrt(lin_mse)
print(" Root mean squared error of linear regression is",lin_rmse)


mean squared error of linear regression is 2.0892465784320273
R Squared score of linear regression is 0.42327985515780187
 Root mean squared error of linear regression is 1.4454226296941761


### cross validation score of linear regression

In [28]:
lin_scores = cross_val_score(lin_reg, housing_prepaired, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores of 10 CV: [1.4196312  1.49072159 1.42353739 1.5343456  1.59586566 1.33995709
 1.40131054 1.42653219 1.41562931 1.41816313]
Mean: 1.4465693687619527
Standard deviation: 0.06984256216643292


### Decision Tree Regression

In [29]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=15,random_state=42)
tree_reg.fit(housing_prepaired, housing_labels)



DecisionTreeRegressor(max_depth=15, random_state=42)

**Evluation metrics **

In [30]:
housing_predictions = tree_reg.predict(housing_prepaired)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
print("mean squared error of Decision tree regressor is",tree_mse)
tree_r2_score=r2_score(housing_labels, housing_predictions)
print("R Squared score of Decision tree regressor is",tree_r2_score)
tree_rmse = np.sqrt(tree_mse)
print(" Root mean squared error of Decision tree regressor is",tree_rmse)


mean squared error of Decision tree regressor is 0.20896905637945629
R Squared score of Decision tree regressor is 0.9423157296477925
 Root mean squared error of Decision tree regressor is 0.4571313338412237


### cross validation score of Decision tree regressor

In [31]:
tree_scores = cross_val_score(tree_reg, housing_prepaired, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-tree_scores)
display_scores(tree_scores)

Scores of 10 CV: [-1.65381331 -1.47035771 -1.58600361 -1.89368216 -1.60933991 -1.39742246
 -1.57376184 -1.50043395 -1.79116544 -1.36043339]
Mean: -1.5836413785712065
Standard deviation: 0.15784016465542702


### Random Forest Regressor

In [32]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(n_estimators=10, random_state=42)
forest_reg.fit(housing_prepaired, housing_labels)



RandomForestRegressor(n_estimators=10, random_state=42)

**Evluation metrics **

In [33]:
housing_predictions = forest_reg.predict(housing_prepaired)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
print("mean squared error of Random forest regressor is",forest_mse)
forest_r2_score=r2_score(housing_labels, housing_predictions)
print("R Squared score of Random forest regressor is",forest_r2_score)
forest_rmse = np.sqrt(forest_mse)
print("Root mean squared error of Random forest regressor is",forest_rmse)


mean squared error of Random forest regressor is 0.1751270334820676
R Squared score of Random forest regressor is 0.9516575548534045
Root mean squared error of Random forest regressor is 0.4184818197748471


### cross validation score of Random Forest Regressor

In [34]:
forest_scores = cross_val_score(forest_reg, housing_prepaired, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

Scores of 10 CV: [0.97672698 1.01042479 0.99516889 1.09412868 0.98043145 0.91732393
 0.99802759 0.96122194 1.00424652 0.95501779]
Mean: 0.989271856474437
Standard deviation: 0.0437609982007956


### Support Vector Regressor

In [35]:
from sklearn.svm import SVR
support=SVR()
support.fit(housing_prepaired, housing_labels)

SVR()

**Evluation metrics **

In [36]:
housing_predictions = support.predict(housing_prepaired)
support_mse = mean_squared_error(housing_labels, housing_predictions)
print("mean squared error of support vector regressor is",support_mse)
support_r2_score=r2_score(housing_labels, housing_predictions)
print("R Squared score of support vector regressor is",support_r2_score)
support_rmse = np.sqrt(support_mse)
print("Root mean squared error of support vector regressor is",support_rmse)


mean squared error of support vector regressor is 2.827887519529216
R Squared score of support vector regressor is 0.21938381199392998
Root mean squared error of support vector regressor is 1.681632397264401


### Cross validation score of Support vector regressor

In [37]:
support_scores = cross_val_score(support, housing_prepaired, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
support_rmse_scores = np.sqrt(-support_scores)
display_scores(support_rmse_scores)

Scores of 10 CV: [1.66941248 1.70502065 1.67638272 1.77619501 1.84212842 1.57480267
 1.63538982 1.69932077 1.65903541 1.67443426]
Mean: 1.691212221054674
Standard deviation: 0.06995248963578975


## 4- Use RandomizedSearchCV to perform fine tuning of the model
that you choose in step 3.  If Support Vector Machine is the model you choose, use the following value range for RandomizedSearchCV. 

i check the r2 score ,MSE and RMSE of all models and conclude that Decision tree is best out of all. So i 
tune the hyperparameters od decision tree using Randomized search CV.

In [38]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
#parameters that need to tune
param_distribs = {
       'max_depth': [3, 5, 10, 20,30],
    'min_samples_leaf': [5, 10, 20, 50, 100,150]
    }



In [39]:
tree_reg = DecisionTreeRegressor(random_state=42)
rnd_search = RandomizedSearchCV(tree_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)

In [40]:
rnd_search.fit(housing_prepaired, housing_labels)

RandomizedSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=42),
                   param_distributions={'max_depth': [3, 5, 10, 20, 30],
                                        'min_samples_leaf': [5, 10, 20, 50, 100,
                                                             150]},
                   random_state=42, scoring='neg_mean_squared_error')

### Tuning results

In [41]:
score_df = pd.DataFrame(rnd_search.cv_results_)
score_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_leaf,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.068767,0.003130,0.001137,0.000041,50,30,"{'min_samples_leaf': 50, 'max_depth': 30}",-1.314427,-1.357587,-1.242706,-1.234223,-1.244909,-1.278771,0.048816,2
1,0.065566,0.001977,0.001185,0.000119,50,10,"{'min_samples_leaf': 50, 'max_depth': 10}",-1.321320,-1.371850,-1.259030,-1.249837,-1.267651,-1.293938,0.046205,3
2,0.053030,0.001645,0.001062,0.000020,150,20,"{'min_samples_leaf': 150, 'max_depth': 20}",-1.613435,-1.591160,-1.499796,-1.452573,-1.547754,-1.540943,0.058865,6
3,0.052199,0.001295,0.001027,0.000021,150,10,"{'min_samples_leaf': 150, 'max_depth': 10}",-1.613435,-1.593421,-1.503146,-1.452573,-1.550167,-1.542548,0.058854,7
4,0.043048,0.003027,0.001058,0.000174,20,5,"{'min_samples_leaf': 20, 'max_depth': 5}",-1.836494,-1.818387,-1.815403,-1.775675,-1.867360,-1.822664,0.029893,9


In [42]:
res=rnd_search.best_estimator_
res

DecisionTreeRegressor(max_depth=10, min_samples_leaf=5, random_state=42)

## 5- Apply the fine-tuned model on test dataset and report the final RMSE

In [43]:
X_test = strat_test_set.drop("median_income", axis=1)
y_test = strat_test_set["median_income"]

In [44]:
X_test['ocean_proximity'] = LabelEncoder().fit_transform(X_test['ocean_proximity'])
X_test['median_house_cat']=LabelEncoder().fit_transform(X_test['median_house_cat'])

In [45]:
sc=StandardScaler(with_mean=False)
X_test_prepaired=sc.fit_transform(X_test)

In [46]:
X_test.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,ocean_proximity,median_house_cat
14645,-117.19,32.77,30.0,2747.0,640.0,3185.0,657.0,3,2
18026,-121.92,37.25,34.0,2231.0,360.0,1035.0,365.0,0,2
17310,-119.50,34.35,39.0,308.0,38.0,59.0,21.0,3,1
14231,-117.04,32.68,9.0,3087.0,609.0,1530.0,556.0,3,0
13227,-117.69,34.01,30.0,2598.0,573.0,2170.0,518.0,1,0


In [47]:
final_predictions_DT = res.predict(X_test_prepaired)
final_mse1 = mean_squared_error(y_test, final_predictions_DT)
print("the mean squared error  ",final_mse1)
final_rmse1 = np.sqrt(final_mse1)
print("root mean squared error ",final_rmse1)
r2_sc=r2_score(y_test, final_predictions_DT)
print("The r2 score is ", r2_sc)

the mean squared error   1.2519562078551245
root mean squared error  1.1189084894910417
The r2 score is  0.6478047944145586


## 6-save your model

In [48]:

joblib.dump(res, "final_model.pkl") 
my_model_loaded = joblib.load("final_model.pkl") 